In [ ]:
from ultralytics import FastSAM, SAM, YOLO
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as img
from pathlib import Path

## Exploring zero shot segmentation models
We will look at the zero shot models and how well they segment an 8 bit RGB Sentinel 2 image over agricultural fields.  
[Starting with FastSAM](https://docs.ultralytics.com/models/fast-sam/)

In [ ]:
model = FastSAM("FastSAM-x.pt")
model.info()
everything_results = model("dogs.jpg", device="cpu", retina_masks=True, imgsz=512, conf=0.4, iou=0.9)
res = everything_results[0]

# Original image as np array
plot_img = res.orig_img.copy()

# Get the masks (N, H, W)
masks = res.masks.data.cpu().numpy()

# Create an overlay for all masks
overlay = plot_img.copy()

# Generate a random color for each mask
for i, mask in enumerate(masks):
    color = np.random.randint(0, 255, (3,), dtype=np.uint8)
    colored_mask = np.stack([mask * c for c in color], axis=-1).astype(np.uint8)
    overlay = cv2.addWeighted(overlay, 1.0, colored_mask, 0.5, 0)

# Plot segmentation masks result
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Segmentation Masks (FastSAM)")
plt.show()

## Testing SAM models on a satellite image
We have already created an RGB tif of a Sentinel 2 image with just red, green, blue bands

In [ ]:
repo_dir = Path.cwd().parent

In [ ]:
s2_img = repo_dir / "data" / "rgb_fast_sam_test.tif"

In [ ]:
# Read the image
img_arr = img.imread(s2_img)

# Plot the image
plt.figure(figsize=(10, 10))
plt.imshow(img_arr)
plt.axis("off")  # Hide axis
plt.title("Image Display")
plt.show()

We can define a common function for extracting and viewing segmentation results as contours

In [ ]:
def plot_segmentation_contours(results, model_name):
    # Get first result (for one image)
    res = results[0]
    img = res.orig_img.copy()

    # Convert to color if grayscale
    if len(img.shape) == 2 or img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    # Get masks
    masks = res.masks.data.cpu().numpy()

    # Draw contours on a copy of the original image
    outline_img = img.copy()

    for mask in masks:
        mask_uint8 = (mask * 255).astype(np.uint8)
        contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        color = tuple(np.random.randint(0, 255, size=3).tolist())
        cv2.drawContours(outline_img, contours, -1, color, thickness=2)

    # Plot
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(outline_img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"Segmentation Outlines {model_name}")
    plt.show()

We will now run FastSAM on the image as is (zero shot no model fine tuning, prompting, input points)

In [ ]:
# Load model and run inference
model = FastSAM("FastSAM-x.pt")
results = model(s2_img, device="cpu", retina_masks=True, imgsz=512, conf=0.3, iou=0.9)

plot_segmentation_contours(results, "FastSAM")


## MobileSAM
MobileSAM takes longer, but produces better results

In [ ]:
# Load MobileSAM
model = SAM("mobile_sam.pt")

results = model(
    s2_img,
    device="cpu",
    imgsz=1024,
    conf=0.3,
    iou=0.9,
)

plot_segmentation_contours(results, "MobileSAM")


The [SAM2t](https://docs.ultralytics.com/models/sam-2/#segment-everything) model does not seem to segment many fields, time taken roughly the same as MobileSAM

In [ ]:
# Load SAM2 tiny
model = SAM("sam2_t.pt")

results = model(
    s2_img,
    device="cpu",
    imgsz=1024,
    conf=0.3,
    iou=0.9,
)

plot_segmentation_contours(results, "SAM2-t")

A YOLO model is extremely fast, does not seem to produce results and must need fine tuning

In [ ]:
model = YOLO("yolov8n-seg.pt")

results = model("rgb_fast_sam_test.tif", device="cpu", imgsz=256, conf=0.1, iou=0.5)

if results[0].masks:

    plot_segmentation_contours(results, "yolov8n")

In conclusion, FastSAM is fast, works on large image, produces quite good results. MobileSAM produces good results, but much slower. SAM2-t does not pick up all areas, might work well with point prompts. YOLO models available from Ultralytics must need fine tuning for this task.